In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
uploaded = files.upload()     #AllNodeAttributes.csv 

Saving AllNodeAttributes.csv to AllNodeAttributes.csv


In [ ]:
!pip install python-igraph
from igraph import *

     |████████████████████████████████| 3.2 MB 7.2 MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.9.8-py3-none-any.whl size=9070 sha256=9329fd6efc1266f239cb09f79758d260c157b2a0320c3f06ed788c56e2b60081
  Stored in directory: /root/.cache/pip/wheels/15/86/ef/b8bcdfbcb1c489771ad256c7cd1eb4971cdb7f3f670938b798
Successfully built python-igraph


In [ ]:
NodeAttributes= pd.read_csv("AllNodeAttributes.csv") 
#NodeAttributes['State'].replace('', np.nan, inplace=True)
#NodeAttributes.dropna(subset=['State'], inplace=True)    #remove all entries where State field is blank #906 users out of 943 left
print("Total nodes are:", NodeAttributes['UserID'].count())
NodeAttributes.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)   #There was an unamed column in the file, needed to delete that column
NodeAttributes.drop(["a"], axis=1, inplace=True)
NodeAttributes = NodeAttributes.drop_duplicates()
print("After dropping duplicates Total nodes are:" , NodeAttributes['UserID'].nunique())
NodeAttributes.head()
NodeAttributes.loc[NodeAttributes['UserID'] == 196]    

#NodeAttributes['GenderType'].loc[NodeAttributes['Gender'] == 1] = 1
#NodeAttributes['GenderType'].loc[NodeAttributes['Gender'] == 0] = 0

NodeAttributes['GenderType'] = np.where(NodeAttributes['Gender'] == 1, 'M','F')

NodeAttributes['AgeGroup'] = NodeAttributes['AgeGroup'].fillna('Teenager')
NodeAttributes['State'] = NodeAttributes['State'].fillna('TX')
print(NodeAttributes['Gender'].value_counts(dropna=False))
NodeAttributes.head()

Total nodes are: 943
After dropping duplicates Total nodes are: 943
1    670
0    273
Name: Gender, dtype: int64


,UserID,Age,Gender,Occupation,Zipcode,NormalizedAge,AgeGroup,Male,Female,State,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,AZ,CA,FL,OH,...,Western,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating,NumMoviesWatched,unknownPreference,ActionPreference,AdventurePreference,AnimationPreference,ChildrenPreference,ComedyPreference,CrimePreference,DocumentaryPreference,DramaPreference,FantasyPreference,FilmNoirPreference,HorrorPreference,MusicalPreference,MysteryPreference,RomancePreference,SciFiPreference,ThrillerPreference,WarPreference,WesternPreference,GenderType
0,1,24,1,technician,85711,0.328767,Youth,1,0,AZ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0.010221,0.666667,0.585714,0.666667,0.440000,0.694505,0.688000,0.96,0.785047,0.7,1.0,0.692308,0.584615,0.720000,0.786364,0.800000,0.723077,0.736000,0.733333,587.0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,M
1,2,53,0,other,94043,0.726027,Senior,0,1,CA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0.000000,0.760000,0.866667,0.800000,0.600000,0.762500,0.755556,0.00,0.765714,0.6,0.9,0.600000,0.600000,0.700000,0.825000,0.750000,0.716667,0.733333,0.000000,123.0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,F
2,3,23,1,writer,32067,0.315068,Youth,1,0,FL,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0.000000,0.557143,0.700000,0.000000,0.000000,0.516667,0.600000,1.00,0.581818,0.0,0.5,0.480000,0.400000,0.636364,0.680000,0.550000,0.504762,0.560000,0.000000,122.0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,M
3,4,24,1,technician,43537,0.328767,Youth,1,0,OH,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0.000000,0.775000,0.700000,0.000000,0.000000,1.000000,0.950000,1.00,0.900000,0.0,0.0,0.800000,1.000000,0.800000,0.866667,0.766667,0.781818,0.900000,0.000000,56.0,0,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,M
4,5,33,0,other,15213,0.452055,MiddleAge,0,1,PA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0.005208,0.628571,0.648485,0.757143,0.489655,0.600000,0.777778,0.00,0.533333,0.5,1.0,0.507143,0.666667,0.600000,0.463158,0.703030,0.589474,0.642857,0.500000,384.0,1,1,1,1,0,1,1,0,0,0,1,0,1,1,0,1,0,1,0,F


In [ ]:
#Creating groups of column names
UnNormalizedFeatures=['Age', 'Occupation',	'Zipcode',	'AgeGroup',	'State','NumMoviesWatched']
NumericFeatures = ['NormalizedAge']
AllStates = pd.unique(NodeAttributes['State']).tolist()
AllStates = [x for x in AllStates if str(x) != 'nan']
AllOccupations = pd.unique(NodeAttributes['Occupation'])
AgeLabels = ['Teenager','Youth','MiddleAge','Senior','SuperSenior']
AllGenres= ['Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller',
       'War', 'Western']
AllGenresRatings = []
for genre in AllGenres:
   AllGenresRatings.append(genre+'Rating')
AllGenresPreference = []
for genre in AllGenres:
   AllGenresPreference.append(genre+'Preference')

In [ ]:
Tier1Columns =  ['Male','Female', 'NormalizedAge'] #['Gender','NormalizedAge'] 
#Tier1Columns.extend(AgeLabels)
Tier1Columns.extend(AllOccupations)
Tier1Columns.extend(AllStates)
print(Tier1Columns)
Tier1Attributes = NodeAttributes[Tier1Columns]
#Tier1Attributes

['Male', 'Female', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor', 'AZ', 'CA', 'FL', 'OH', 'PA', 'WA', 'VT', 'MA', 'GA', 'CT', 'SC', 'MN', 'OR', 'NY', 'TN', 'KY', 'MI', 'MD', 'TX', 'NC', 'WI', 'CO', 'IN', 'IA', 'NJ', 'UT', 'IL', 'VA', 'OK', 'NH', 'DC', 'NV', 'NE', 'LA', 'MO', 'KS', 'AK', 'AL', 'ID', 'ND', 'MS', 'AR', 'MT', 'NM', 'RI', 'HI', 'WV', 'ME', 'DE', 'WY', 'SD']


In [ ]:
#from sklearn.metrics.pairwise import pairwise_distances
#distance = pairwise_distances(NodeAttributes,metric='euclidean')
#maxValue = np.amax(distance)
#print(maxValue)
#similarityMatrix = 1 - distance/maxValue


from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
similarityMatrix = cosine_similarity(Tier1Attributes)     #cosine_similarity(Tier1Attributes, Tier1Attributes)
np.fill_diagonal(similarityMatrix, 0)    #to remove self loops
print('mean median mode: ',similarityMatrix.mean(), np.median(similarityMatrix),stats.mode(similarityMatrix,axis=None))
print(similarityMatrix.shape)
#similarityMatrix

mean median mode:  0.2942004718747698 0.3605351773774652 ModeResult(mode=array([0.36294844]), count=array([1656]))
(943, 943)


In [ ]:
thresh = 0.5 #np.median(similarityMatrix) #0.5                  
#values_morethanequalto_thresh = (cosinematrix >= thresh)
#values_morethanequalto_thresh.sum()
adjacencymatrix = similarityMatrix.copy()
adjacencymatrix[adjacencymatrix >= thresh] = 1
adjacencymatrix[adjacencymatrix < thresh] = 0
adjacencymatrix
values_equalto_one = (adjacencymatrix == 1)
values_equalto_one.sum()

74574

In [ ]:
#print( NodeAttributes.UserID )
print("Total nodes are:", NodeAttributes['UserID'].count())
print(adjacencymatrix.shape)
graph = Graph.Adjacency(adjacencymatrix, mode="undirected")  
graph.vs['name'] = NodeAttributes.UserID 
graph.vs['Age'] = NodeAttributes.Age 
graph.vs['Gender'] = NodeAttributes.Gender
Occupation_codes = NodeAttributes.Occupation.astype('category').cat.codes  #converting to numeric category
graph.vs['Occupation'] = Occupation_codes
Age_codes = NodeAttributes.AgeGroup.astype('category').cat.codes #converting to numeric category
graph.vs['AgeGroup'] = Age_codes
State_codes = NodeAttributes.State.astype('category').cat.codes  #converting to numeric category
graph.vs['State'] = State_codes
Gender_codes = NodeAttributes.GenderType.astype('category').cat.codes  #converting to numeric category
graph.vs['GenderType'] = Gender_codes

print(graph.vcount(),graph.ecount())
graph=graph.simplify()
print(graph.vcount(),graph.ecount())
#summary(graph)
len(graph.vs['AgeGroup'])

Total nodes are: 943
(943, 943)
943 37287
943 37287


943

In [ ]:
print('User Features for Tier1: ',Tier1Columns)
print(len(Tier1Columns))
communities = Graph.community_multilevel(graph)
membership = communities.membership
print('mean median mode: ',similarityMatrix.mean(), np.median(similarityMatrix),stats.mode(similarityMatrix,axis=None))
print('thresh = ', thresh)
print('vcount ecount: ',graph.vcount(),graph.ecount())
print('modularity: ', graph.modularity(communities))
print('Number of communities: ',len(communities))
for n in range(0,len(communities)):
  print ('Community nº', n, 'size:', len(communities[n]))

User Features for Tier1:  ['Male', 'Female', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor', 'AZ', 'CA', 'FL', 'OH', 'PA', 'WA', 'VT', 'MA', 'GA', 'CT', 'SC', 'MN', 'OR', 'NY', 'TN', 'KY', 'MI', 'MD', 'TX', 'NC', 'WI', 'CO', 'IN', 'IA', 'NJ', 'UT', 'IL', 'VA', 'OK', 'NH', 'DC', 'NV', 'NE', 'LA', 'MO', 'KS', 'AK', 'AL', 'ID', 'ND', 'MS', 'AR', 'MT', 'NM', 'RI', 'HI', 'WV', 'ME', 'DE', 'WY', 'SD']
75
mean median mode:  0.2942004718747698 0.3605351773774652 ModeResult(mode=array([0.36294844]), count=array([1656]))
thresh =  0.5
vcount ecount:  943 37287
modularity:  0.5621249263305587
Number of communities:  7
Community nº 0 size: 204
Community nº 1 size: 266
Community nº 2 size: 131
Community nº 3 size: 136
Community nº 4 size: 67
Community nº 5 size: 59
Community nº

User Features:  ['Male', 'Female', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor']
mean median mode:  0.3998545134182831 0.5251757751693631 ModeResult(mode=array([0.52433589]), count=array([1734]))
thresh =  0.5
vcount ecount:  943 276370
modularity:  0.2146715460302999
Number of communities:  2
Community nº 0 size: 669
Community nº 1 size: 274




User Features:  ['Gender', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor']
mean median mode:  0.39143422958850677 0.5214861679327384 ModeResult(mode=array([0.52775383]), count=array([1482]))
thresh =  0.5
vcount ecount:  943 243480
modularity:  0.07772386520763802
Number of communities:  8
Community nº 0 size: 399
Community nº 1 size: 105
Community nº 2 size: 79
Community nº 3 size: 197
Community nº 4 size: 95
Community nº 5 size: 51
Community nº 6 size: 6
Community nº 7 size: 11


User Features:  ['Gender', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor', 'AZ', 'CA', 'FL', 'OH', 'PA', 'WA', 'VT', 'MA', 'GA', 'CT', 'SC', 'MN', 'OR', 'NY', 'TN', 'KY', 'MI', 'MD', 'TX', 'NC', 'WI', 'CO', 'IN', 'IA', 'NJ', 'UT', 'IL', 'VA', 'OK', 'NH', 'DC', 'NV', 'NE', 'LA', 'MO', 'KS', 'AK', 'AL', 'ID', 'ND', 'MS', 'AR', 'MT', 'NM', 'RI', 'HI', 'WV', 'ME', 'DE', 'WY', 'SD']
mean median mode:  0.281512575287697 0.3575089592191391 ModeResult(mode=array([0.36294844]), count=array([1336]))
thresh =  0.5
vcount ecount:  943 38947
modularity:  0.5440391549378877
Number of communities:  8
Community nº 0 size: 117
Community nº 1 size: 107
Community nº 2 size: 156
Community nº 3 size: 179
Community nº 4 size: 139
Community nº 5 size: 146
Community nº 6 size: 26
Community nº 7 size: 73


User Features for Tier1:  ['Gender', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor', 'AZ', 'CA', 'FL', 'OH', 'PA', 'WA', 'VT', 'MA', 'GA', 'CT', 'SC', 'MN', 'OR', 'NY', 'TN', 'KY', 'MI', 'MD', 'TX', 'NC', 'WI', 'CO', 'IN', 'IA', 'NJ', 'UT', 'IL', 'VA', 'OK', 'NH', 'DC', 'NV', 'NE', 'LA', 'MO', 'KS', 'AK', 'AL', 'ID', 'ND', 'MS', 'AR', 'MT', 'NM', 'RI', 'HI', 'WV', 'ME', 'DE', 'WY', 'SD']
mean median mode:  0.281512575287697 0.3575089592191391 ModeResult(mode=array([0.36294844]), count=array([1336]))
thresh =  0.5
vcount ecount:  943 38947
modularity:  0.5495987382386739
Number of communities:  9
Community nº 0 size: 160
Community nº 1 size: 113
Community nº 2 size: 122
Community nº 3 size: 146
Community nº 4 size: 103
Community nº 5 size: 137
Community nº 6 size: 26
Community nº 7 size: 60
Community nº 8 size: 76



In [ ]:
for nodes_in_community in communities:
  subgraph = graph.subgraph(nodes_in_community)
  #print(subgraph.vs['Gender'])
  print('vcount ecount: ',subgraph.vcount(),subgraph.ecount())
  ag = GraphBase.assortativity_nominal(subgraph, subgraph.vs["Gender"], directed = 'FALSE')
  agt = GraphBase.assortativity_nominal(subgraph, subgraph.vs["GenderType"], directed = 'FALSE')
  ao = GraphBase.assortativity_nominal(subgraph, subgraph.vs["Occupation"], directed = 'FALSE')
  aa = GraphBase.assortativity_nominal(subgraph, subgraph.vs["AgeGroup"], directed = 'FALSE')
  ast = GraphBase.assortativity_nominal(subgraph, subgraph.vs["State"], directed = 'FALSE')
  print('assortativity: Gender: ', ag, ' GenderType: ', agt,' Occupation: ', ao, ' Age: ', aa, ' State:', ast)

vcount ecount:  204 2747
assortativity: Gender:  nan  GenderType:  nan  Occupation:  0.7001758063834956  Age:  0.04215136435398537  State: 0.21649781199865153
vcount ecount:  266 5326
assortativity: Gender:  nan  GenderType:  nan  Occupation:  0.7210277380134139  Age:  0.14816762783544785  State: 0.2077271197064147
vcount ecount:  131 4970
assortativity: Gender:  0.19919436052361725  GenderType:  0.19919436052361725  Occupation:  0.23682644179062093  Age:  -0.007786583051585672  State: 0.28953168270886553
vcount ecount:  136 9180
assortativity: Gender:  nan  GenderType:  nan  Occupation:  nan  Age:  -0.0074074074074073825  State: -0.0074074074074074224
vcount ecount:  67 1971
assortativity: Gender:  nan  GenderType:  nan  Occupation:  0.31518906711448913  Age:  -0.016090168250232975  State: -0.008848880627703839
vcount ecount:  59 1479
assortativity: Gender:  0.33172413793103456  GenderType:  0.33172413793103456  Occupation:  nan  Age:  -0.02046770598701997  State: 0.004401144267942023

In [ ]:
#for z in communities[1][:10]:
 # print(graph.vs[z]['name'])
#print(type(membership))
#print(type(graph.vs["name"]))

In [ ]:
membershipdata = zip(graph.vs["name"], membership)
CommunityMembership =  pd.DataFrame(membershipdata, index=range(0,graph.vcount()), columns=["UserID","Community"])
CommunityMembership.head() 
#CommunityMembership.loc[CommunityMembership['UserID'] == 196] 

,UserID,Community
0,1,0
1,2,1
2,3,0
3,4,0
4,5,1


In [ ]:
def edge_support(G):
    neighbors=G.neighborhood()    #neighbors_iter
    nbrs=dict((v.index,set(neighbors[v.index])) for v in G.vs)
    support = {}
    for e in G.es:
        nod1,nod2 = e.source, e.target	
        nod1_nbrs = set(nbrs[nod1])-set([nod1])
        nod2_nbrs = set(nbrs[nod2])-set([nod2])
        sup = len(nod1_nbrs.intersection(nod2_nbrs))
        #G[nod1][nod2]['support'] = sup
        support[(nod1,nod2)] = sup
    #print 'support :', support
    return support

def ktruss(G):   
    #G = G.simplify()    #assume graph is simple
    #G.to_undirected(mode=False)
    support = edge_support(G)
    edges=sorted(support,key=support.get)
    bin_boundaries=[0]
    curr_support=0
    for i,e in enumerate(edges):
        if support[e]>curr_support:
            bin_boundaries.extend([i]*(support[e]-curr_support))
            curr_support=support[e]
            
    edge_pos = dict((e,pos) for pos,e in enumerate(edges))
    #print 'edge-pos:',edge_pos
    truss={}         ## initial guesses for truss is support
    neighbors=G.neighborhood()    #neighbors_iter
    #print 'neighbors:', neighbors
    nbrs=dict((v.index,(set(neighbors[v.index])-set([v.index]))) for v in G.vs)
    #nbrs=dict((v.index,set(neighbors[v.index])) for v in G.vs)
    #print 'nbrs:', nbrs
    for e in edges:
      #print 'processing edge : ', e, 'support :', support[e], 'pos:', edge_pos[e]
      u,v =e[0], e[1]
      if not(u == v) :
        common_nbrs = set(nbrs[u]).intersection(nbrs[v])
        #print u,v,'common_nbrs',common_nbrs
        for w in common_nbrs:
            if (u,w) in support :             
               e1 = (u,w)
            else :
               e1 = (w,u)
            if (v,w) in support :
               e2 = (v,w)
            else:
               e2 = (w,v)
            pos=edge_pos[e1]
            if support[e1] > support[e] :
               bin_start=bin_boundaries[support[e1]]
               edge_pos[e1]=bin_start
               edge_pos[edges[bin_start]]=pos
               edges[bin_start],edges[pos]=edges[pos],edges[bin_start]
               bin_boundaries[support[e1]]+=1
            #print 'e1',e1,'support:',support[e1], 'pos:', pos, 'new pos:', edge_pos[e1]
            
            pos=edge_pos[e2]
            if support[e2] > support[e] :
               bin_start=bin_boundaries[support[e2]]
               edge_pos[e2]=bin_start
               edge_pos[edges[bin_start]]=pos
               edges[bin_start],edges[pos]=edges[pos],edges[bin_start]
               bin_boundaries[support[e2]]+=1
            #print 'e2',e2,'support:',support[e2], 'pos:', pos, 'new pos:', edge_pos[e2]
              
            support[e1] =  max(support[e], support[e1]-1)     
            support[e2] =  max(support[e], support[e2]-1)   
            
        truss[e] = support[e] + 2 
        nbrs[u].remove(v)
        nbrs[v].remove(u)
    #print 'Truss: ', truss
    #print 'Sorted Truss: ', sorted(truss,key=truss.get)
    return truss

def getnodetrussness(graph):
    n = graph.vcount()
    ktrussdict = ktruss(graph)
    nodetruss = [0] * n
    for edge in graph.es:
      source = edge.source		
      target = edge.target
      if not (source == target) :
         t = ktrussdict[(source,target)]
      else:
         t = 0		  
      nodetruss[source] = max(nodetruss[source], t)
      nodetruss[target] = max(nodetruss[target], t)
    
    return nodetruss 

#trussness =  getnodetrussness(graph)
#trussness

In [ ]:
CommunityMembershipCores = pd.DataFrame(columns=["UserID","T1CorenessinCommunity"]);

for nodes_in_community in communities:
  subgraph = graph.subgraph(nodes_in_community)
  #print('vcount ecount: ',subgraph.vcount(),subgraph.ecount())
  coreness = GraphBase.coreness(subgraph)
  trussness =  getnodetrussness(graph)
  corenessdata = zip(subgraph.vs["name"], coreness,trussness)

  df = pd.DataFrame(corenessdata, index=range(0,subgraph.vcount()), columns=["UserID","T1CorenessinCommunity","T1TrussinCommunity"])
  frames = [CommunityMembershipCores, df]
  CommunityMembershipCores =  pd.concat(frames)

CommunityData = pd.merge(left=CommunityMembership, right=CommunityMembershipCores, how='left', left_on='UserID', right_on='UserID') 
CommunityData.loc[CommunityData['UserID'] == 198]

,UserID,Community,T1CorenessinCommunity,T1TrussinCommunity
197,198,1,59,60.0


In [ ]:
NodeAttributesWithCommunity = pd.merge(left=CommunityMembership, right=NodeAttributes, how='left', left_on='UserID', right_on='UserID') 
NodeAttributesWithCommunity.loc[NodeAttributesWithCommunity['UserID'] == 1]
NodeAttributesWithCommunity

#SortedNodeAttributesWithCommunity = NodeAttributesWithCommunity.sort_values(by="Community", ascending=True)
#SortedNodeAttributesWithCommunity

,UserID,Community,Age,Gender,Occupation,Zipcode,NormalizedAge,AgeGroup,Male,Female,State,Teenager,Youth,MiddleAge,Senior,SuperSenior,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,AZ,CA,FL,...,Western,ActionRating,AdventureRating,AnimationRating,ChildrenRating,ComedyRating,CrimeRating,DocumentaryRating,DramaRating,FantasyRating,FilmNoirRating,HorrorRating,MusicalRating,MysteryRating,RomanceRating,SciFiRating,ThrillerRating,WarRating,WesternRating,NumMoviesWatched,unknownPreference,ActionPreference,AdventurePreference,AnimationPreference,ChildrenPreference,ComedyPreference,CrimePreference,DocumentaryPreference,DramaPreference,FantasyPreference,FilmNoirPreference,HorrorPreference,MusicalPreference,MysteryPreference,RomancePreference,SciFiPreference,ThrillerPreference,WarPreference,WesternPreference,GenderType
0,1,0,24,1,technician,85711,0.328767,Youth,1,0,AZ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...,0.010221,0.666667,0.585714,0.666667,0.440000,0.694505,0.688000,0.96,0.785047,0.7,1.0,0.692308,0.584615,0.720000,0.786364,0.800000,0.723077,0.736000,0.733333,587.0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,M
1,2,1,53,0,other,94043,0.726027,Senior,0,1,CA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.760000,0.866667,0.800000,0.600000,0.762500,0.755556,0.00,0.765714,0.6,0.9,0.600000,0.600000,0.700000,0.825000,0.750000,0.716667,0.733333,0.000000,123.0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,F
2,3,0,23,1,writer,32067,0.315068,Youth,1,0,FL,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,...,0.000000,0.557143,0.700000,0.000000,0.000000,0.516667,0.600000,1.00,0.581818,0.0,0.5,0.480000,0.400000,0.636364,0.680000,0.550000,0.504762,0.560000,0.000000,122.0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,M
3,4,0,24,1,technician,43537,0.328767,Youth,1,0,OH,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0.000000,0.775000,0.700000,0.000000,0.000000,1.000000,0.950000,1.00,0.900000,0.0,0.0,0.800000,1.000000,0.800000,0.866667,0.766667,0.781818,0.900000,0.000000,56.0,0,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,M
4,5,1,33,0,other,15213,0.452055,MiddleAge,0,1,PA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0.005208,0.628571,0.648485,0.757143,0.489655,0.600000,0.777778,0.00,0.533333,0.5,1.0,0.507143,0.666667,0.600000,0.463158,0.703030,0.589474,0.642857,0.500000,384.0,1,1,1,1,0,1,1,0,0,0,1,0,1,1,0,1,0,1,0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,939,1,26,0,student,33319,0.356164,Youth,0,1,FL,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0.000000,0.811111,0.700000,0.800000,0.800000,0.906667,0.866667,0.00,0.911111,0.8,0.0,0.600000,0.800000,0.800000,0.960000,0.825000,0.816667,1.000000,0.000000,103.0,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,0,F
939,940,0,32,1,administrator,2215,0.438356,MiddleAge,1,0,MA,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.686957,0.600000,0.900000,0.840000,0.704762,0.857143,0.00,0.720833,0.0,0.8,0.600000,0.628571,0.866667,0.708333,0.610526,0.670000,0.653333,0.000000,234.0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0,M
940,941,3,20,1,student,97229,0.273973,Youth,1,0,OR,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0.000000,0.760000,0.771429,0.933333,0.900000,0.828571,0.600000,0.00,0.840000,0.0,0.0,0.000000,0.800000,1.000000,1.000000,0.775000,0.800000,1.000000,0.000000,54.0,0,1,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,0,M
941,942,1,48,0,librarian,78209,0.657534,MiddleAge,0,1,TX,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.017143,0.822222,0.945455,0.950000,0.861538,0.818182,0.000000,0.00

In [ ]:
CommunityCentroids = NodeAttributesWithCommunity.groupby(by=["Community"],as_index=False).mean()
CommunityCentroids

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

#Tier2columns = AllGenresPreference
#Tier2columns.extend(AllGenres)
#Tier2columns.exted(AllGenresRatings)   
#print(Tier2columns)
#NodesInCommunity = NodeAttributesWithCommunity.loc[NodeAttributesWithCommunity['Community'] == 0][Tier2columns]
#NodesInCommunity  #160 rows × 72 columns

In [ ]:
communityMembers = NodeAttributesWithCommunity.groupby('Community')['UserID'].apply(list).to_dict()

Tier2Columns = []
Tier2Columns.extend(AllGenres)
#Tier2Columns.extend(AllGenresPreference)
#Tier2Columns.extend(AllGenresRatings)      #with oly genreratings no community is split
print('Tier1 Features: ',Tier1Columns)  
print('Tier2 Features: ',Tier2Columns) 
SubCommunityMembership= pd.DataFrame(columns=["UserID","SubCommunity"]);
numCommunties = len(communities)
T2CommunityMembershipCores = pd.DataFrame(columns=["UserID","T2CorenessinCommunity","T2TrussinCommunity"]);
for communitylabel in communityMembers:
  NodesInCommunity = NodeAttributesWithCommunity.loc[NodeAttributesWithCommunity['Community'] == communitylabel][Tier2Columns]
  cosineSimilarity = cosine_similarity(NodesInCommunity)
  thresh = 0.5
  adjacencyMatrix = cosineSimilarity.copy()
  adjacencyMatrix[adjacencyMatrix >= thresh] = 1
  adjacencyMatrix[adjacencyMatrix < thresh] = 0
  adjacencyMatrix_df = pd.DataFrame(adjacencyMatrix, columns=NodesInCommunity.index, index=NodesInCommunity.index)
  subgraph = Graph.Adjacency(adjacencyMatrix, mode="undirected")  
  subgraph.vs['name'] = communityMembers[communitylabel]      #NodeAttributesWithCommunity.UserID 
  print(subgraph.vcount(),subgraph.ecount())
  subgraph=subgraph.simplify()
  subcommunities = Graph.community_multilevel(subgraph)
  submembership = subcommunities.membership
  print('mean median mode: ',cosineSimilarity.mean(), np.median(cosineSimilarity),stats.mode(cosineSimilarity,axis=None))
  print('thresh = ', thresh)
  print('vcount ecount: ',subgraph.vcount(),subgraph.ecount())
  print('modularity: ', subgraph.modularity(subcommunities))
  print('Number of subcommunities: ',len(subcommunities))
  for n in range(0,len(subcommunities)):
    print ('Community nº', n, 'size:', len(subcommunities[n]))

  for nodes_in_subcommunity in subcommunities:
  
    subsubgraph = subgraph.subgraph(nodes_in_subcommunity)
    print('subsubgraph vcount ecount: ',subsubgraph.vcount(),subsubgraph.ecount())
    coreness = GraphBase.coreness(subsubgraph)
    trussness =  getnodetrussness(subsubgraph)
    #print('coreness: ',coreness)
    corenessdata = zip(subsubgraph.vs["name"], coreness,trussness)
    df = pd.DataFrame(corenessdata, index=range(0,subsubgraph.vcount()), columns=["UserID","T2CorenessinCommunity","T2TrussinCommunity"])
    frames = [T2CommunityMembershipCores, df]
    T2CommunityMembershipCores =  pd.concat(frames)
    #print(T2CommunityMembershipCores.loc[T2CommunityMembershipCores['UserID'] == 5])

  submembershipdata = zip(subgraph.vs["name"], submembership)
  df = pd.DataFrame(submembershipdata, index=range(0,subgraph.vcount()), columns=["UserID","SubCommunity"])
  frames = [SubCommunityMembership, df]
  SubCommunityMembership =  pd.concat(frames)
  print('Num SubCommunityMembership rows:', df.shape)
  #print(SubCommunityMembership.tail())

SubCommunityData = pd.merge(left=SubCommunityMembership, right=T2CommunityMembershipCores, how='left', left_on='UserID', right_on='UserID') 
SubCommunityData.loc[SubCommunityData['UserID'] == 5]
T2CommunityMembershipCores.loc[T2CommunityMembershipCores['UserID'] == 5]


Tier1 Features:  ['Male', 'Female', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor', 'AZ', 'CA', 'FL', 'OH', 'PA', 'WA', 'VT', 'MA', 'GA', 'CT', 'SC', 'MN', 'OR', 'NY', 'TN', 'KY', 'MI', 'MD', 'TX', 'NC', 'WI', 'CO', 'IN', 'IA', 'NJ', 'UT', 'IL', 'VA', 'OK', 'NH', 'DC', 'NV', 'NE', 'LA', 'MO', 'KS', 'AK', 'AL', 'ID', 'ND', 'MS', 'AR', 'MT', 'NM', 'RI', 'HI', 'WV', 'ME', 'DE', 'WY', 'SD']
Tier2 Features:  ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western']
204 20646
mean median mode:  0.854674883293243 0.8844445412194741 ModeResult(mode=array([1.]), count=array([60]))
thresh =  0.5
vcount ecount:  204 20442
modularity:  0.00

NameError: ignored

In [ ]:
#for Tier-II Assortativity

communityMembers = NodeAttributesWithCommunity.groupby('Community')['UserID'].apply(list).to_dict()

Tier2Columns = []
Tier2Columns.extend(AllGenres)
#Tier2Columns.extend(AllGenresPreference)
#Tier2Columns.extend(AllGenresRatings)      #with only genreratings no community is split
print('Tier1 Features: ',Tier1Columns)  
print('Tier2 Features: ',Tier2Columns) 

numCommunties = len(communities)
for communitylabel in communityMembers:
  NodesInCommunity = NodeAttributesWithCommunity.loc[NodeAttributesWithCommunity['Community'] == communitylabel][Tier2Columns]
  cosineSimilarity = cosine_similarity(NodesInCommunity)
  thresh = 0.5
  adjacencyMatrix = cosineSimilarity.copy()
  adjacencyMatrix[adjacencyMatrix >= thresh] = 1
  adjacencyMatrix[adjacencyMatrix < thresh] = 0
  adjacencyMatrix_df = pd.DataFrame(adjacencyMatrix, columns=NodesInCommunity.index, index=NodesInCommunity.index)
  subgraph = Graph.Adjacency(adjacencyMatrix, mode="undirected")  
  subgraph.vs['name'] = communityMembers[communitylabel]      #communityMembers is a dict of community members
  subgraph=subgraph.simplify()
  subcommunities = Graph.community_multilevel(subgraph)
  submembership = subcommunities.membership

  for nodes_in_subcommunity in subcommunities:
    SubGraphNodeAttributes = NodeAttributes[NodeAttributes['UserID'].isin(nodes_in_subcommunity)]
    subsubgraph = subgraph.subgraph(nodes_in_subcommunity)
    print('subsubgraph vcount ecount: ',subsubgraph.vcount(),subsubgraph.ecount())

    subsubgraph.vs['Age'] =  list(SubGraphNodeAttributes.Age )
    subsubgraph.vs['Gender'] = list(SubGraphNodeAttributes.Gender)
    print(subsubgraph.vs['Gender'] )
    Occupation_codes = SubGraphNodeAttributes.Occupation.astype('category').cat.codes  #converting to numeric category
    subsubgraph.vs['Occupation'] = list(Occupation_codes)
    Age_codes = SubGraphNodeAttributes.AgeGroup.astype('category').cat.codes #converting to numeric category
    subsubgraph.vs['AgeGroup'] = list(Age_codes)
    State_codes = SubGraphNodeAttributes.State.astype('category').cat.codes  #converting to numeric category
    subsubgraph.vs['State'] =list(State_codes)
    

    ag = GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs["Gender"], directed = 'FALSE')
    ao = GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs["Occupation"], directed = 'FALSE')
    aa = GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs["AgeGroup"], directed = 'FALSE')
    ast = GraphBase.assortativity_nominal(subsubgraph, subsubgraph.vs["State"], directed = 'FALSE')
    print('assortativity: Gender: ', ag, ' Occupation: ', ao, ' Age: ', aa, ' State:', ast)
   
    print(subsubgraph.vs['Gender'])
    

Tier1 Features:  ['Male', 'Female', 'NormalizedAge', 'technician', 'other', 'writer', 'executive', 'administrator', 'student', 'lawyer', 'educator', 'scientist', 'entertainment', 'programmer', 'librarian', 'homemaker', 'artist', 'engineer', 'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor', 'AZ', 'CA', 'FL', 'OH', 'PA', 'WA', 'VT', 'MA', 'GA', 'CT', 'SC', 'MN', 'OR', 'NY', 'TN', 'KY', 'MI', 'MD', 'TX', 'NC', 'WI', 'CO', 'IN', 'IA', 'NJ', 'UT', 'IL', 'VA', 'OK', 'NH', 'DC', 'NV', 'NE', 'LA', 'MO', 'KS', 'AK', 'AL', 'ID', 'ND', 'MS', 'AR', 'MT', 'NM', 'RI', 'HI', 'WV', 'ME', 'DE', 'WY', 'SD']
Tier2 Features:  ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western']
subsubgraph vcount ecount:  108 5766
[1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
for nodes_in_community in communities:
  subgraph = graph.subgraph(nodes_in_community)
  #print(subgraph.vs['Occupation'])
  #print('vcount ecount: ',subgraph.vcount(),subgraph.ecount())
  ag = GraphBase.assortativity_nominal(subgraph, subgraph.vs["Gender"], directed = 'FALSE')
  ao = GraphBase.assortativity_nominal(subgraph, subgraph.vs["Occupation"], directed = 'FALSE')
  aa = GraphBase.assortativity_nominal(subgraph, subgraph.vs["AgeGroup"], directed = 'FALSE')
  ast = GraphBase.assortativity_nominal(subgraph, subgraph.vs["State"], directed = 'FALSE')
  print('assortativity: Gender: ', a, ' Occupation: ', ao, ' Age: ', aa, ' State:', ast)

In [ ]:
CommunityData
T1T2CommunityCores = pd.merge(left=CommunityData, right=SubCommunityData, how='left', left_on='UserID', right_on='UserID') 
T1T2CommunityCores["ComWithSubCom"] = T1T2CommunityCores["Community"].astype(str) + T1T2CommunityCores["SubCommunity"].astype(str)
T1T2CommunityCores.loc[T1T2CommunityCores['UserID'] == 5]

,UserID,Community,T1CorenessinCommunity,T1TrussinCommunity,SubCommunity,T2CorenessinCommunity,T2TrussinCommunity,ComWithSubCom
4,5,1,38,39.0,1,49,50,11


In [ ]:
SubCommunityMembership.loc[SubCommunityMembership['UserID']==196]

In [ ]:
membershipdata = zip(graph.vs["name"], membership)
CommunityMembership =  pd.DataFrame(membershipdata, index=range(0,graph.vcount()), columns=["UserID","Community"])
CommunityMembership.head() 
CommunityMembership.loc[CommunityMembership['UserID'] == 196] 


NodeAttributesWithSubCommunity = pd.merge(left=SubCommunityMembership,right=NodeAttributesWithCommunity,  how='left', left_on='UserID', right_on='UserID')
NodeAttributesWithSubCommunity["ComWithSubCom"] = NodeAttributesWithSubCommunity["Community"].astype(str) + NodeAttributesWithSubCommunity["SubCommunity"].astype(str)
print(NodeAttributesWithSubCommunity.shape)
print("Total nodes are:" , NodeAttributes['UserID'].nunique())
NodeAttributesWithSubCommunity.tail()
NodeAttributesWithSubCommunity.loc[NodeAttributesWithSubCommunity['UserID']==196]

In [ ]:
SubCommunityCentroids = NodeAttributesWithSubCommunity.groupby(by=["ComWithSubCom"],as_index=False).mean()
SubCommunityCentroids

In [ ]:
columnsToKeep = ['ComWithSubCom','Gender']
columnsToKeep.extend(AgeLabels)
columnsToKeep.extend(AllOccupations)
print(columnsToKeep)
UserDemography = NodeAttributesWithSubCommunity[columnsToKeep]
UserDemographyCentroids = UserDemography.groupby(by=["ComWithSubCom"],as_index=False).mean()
print(UserDemographyCentroids.shape)
UserDemographyCentroids

In [ ]:
num_users = UserDemography.shape[0]
num_centroids = UserDemographyCentroids.shape[0]

centroidNames = dict(list(zip(UserDemographyCentroids.index, UserDemographyCentroids.ComWithSubCom)))
#print(centroidNames)
assignedCentroids=[]
for i in range(0,num_users):
  row = UserDemography.iloc[i].to_numpy().reshape(1, -1)
  maxsimilarity = 0
  matchedcentroid = 0
  for j in range(0,num_centroids):
    comparedrow = UserDemographyCentroids.iloc[j].to_numpy().reshape(1, -1)
    sim = cosine_similarity(row,comparedrow).item()   #converting to scalar
    #print(sim)
    if (sim > maxsimilarity):
      maxsimilarity = sim
      matchedcentroid = centroidNames[j]
  assignedCentroids.append(matchedcentroid)

In [ ]:
columnsToKeep = ['UserID','ComWithSubCom']
MatchedCommunity = NodeAttributesWithSubCommunity[columnsToKeep]
MatchedCommunity['AssignedCommunity'] = assignedCentroids
MatchedCommunity.tail()
#MatchedCommunity.to_csv("/drive/My Drive/Colab Notebooks/MovieLens-recommender/matchedsubcommunities.csv")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_test = MatchedCommunity['ComWithSubCom']
y_pred = MatchedCommunity['AssignedCommunity']
print(precision_score(y_test, y_pred, labels=np.unique(y_pred), average="weighted"))
print(recall_score(y_test, y_pred, labels=np.unique(y_pred), average="weighted"))   

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
T1T2CommunityCores.to_csv("/drive/My Drive/Colab Notebooks/MovieLens-recommender/Data/T1T2CommunityCoresTrusses.csv")

In [ ]:
#NodeAttributes.to_csv("/drive/My Drive/Colab Notebooks/MovieLens-recommender/AllNodeAttributes.csv")

In [ ]:
NodeAttributesWithSubCommunity.to_csv("/drive/My Drive/Colab Notebooks/MovieLens-recommender/Data/NodeAttributesWithSubCommunityGC.csv")

In [ ]:
#CommunityCentroids.to_csv("/drive/My Drive/Colab Notebooks/MovieLens-recommender/CommunityCentroids.csv")

In [ ]:
#MatchedCommunity.to_csv("/drive/My Drive/Colab Notebooks/MovieLens-recommender/MatchedCommunity.csv")

In [ ]:
'''
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
'''

In [ ]:
#pip install cairocffi
import cairocffi

In [ ]:
!apt-get install libcairo2-dev libjpeg-dev libgif-dev
!pip install pycairo

In [ ]:
import cairo
plot(communities, mark_groups = True)